# 카카오 로컬 API: 도로명 주소 --> 위도, 경도

In [1]:
from google.colab import files
uploaded= files.upload()
filename= list(uploaded.keys())[0]
filename

Saving kakaoapikey.txt to kakaoapikey.txt


'kakaoapikey.txt'

In [2]:
with open(filename) as f:
  api_key = f.read()
len(api_key)

32

In [3]:
import requests
from urllib.parse import quote

In [4]:
addr = '서울특별시 중구 태평로1가 31 서울특별시청'
search_url = "https://dapi.kakao.com/v2/local/search/address.json" 
url = f'{search_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A4%91%EA%B5%AC%20%ED%83%9C%ED%8F%89%EB%A1%9C1%EA%B0%80%2031%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EC%B2%AD'

In [5]:
# "Authorization: KakaoAK {REST_API_KEY}"
result= requests.get(url,
                headers={"Authorization": f'KakaoAK {api_key}'}).json()

result          

{'documents': [{'address': {'address_name': '서울 중구 태평로1가 31',
    'b_code': '1114010300',
    'h_code': '1114055000',
    'main_address_no': '31',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '명동',
    'region_3depth_name': '태평로1가',
    'sub_address_no': '',
    'x': '126.978203640984',
    'y': '37.566585446882'},
   'address_name': '서울 중구 태평로1가 31',
   'address_type': 'REGION_ADDR',
   'road_address': {'address_name': '서울 중구 세종대로 110',
    'building_name': '서울특별시청',
    'main_building_no': '110',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '태평로1가',
    'road_name': '세종대로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.977829174031',
    'y': '37.5663174209601',
    'zone_no': '04524'},
   'x': '126.978203640984',
   'y': '37.566585446882'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [6]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])

lng,lat

(126.978203640984, 37.566585446882)

## 공공기관.csv파일에 위도,경도 추가하기

In [7]:
uploaded= files.upload()
filename= list(uploaded.keys())[0]

Saving 공공기관.csv to 공공기관.csv


In [8]:
import pandas as pd

df = pd.read_csv(filename)
df

,공공기관,도로명주소
0,종로구청,서울특별시 종로구 종로1길 36(수송동)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,양천구청,서울특별시 양천구 목동동로 105(신정동)


In [10]:
lng_list,lat_list= [ ],[ ]

for i in df['도로명주소']:
  url = f'{search_url}?query={i}'
  result= requests.get(url,
                headers={"Authorization": f'KakaoAK {api_key}'}).json()
  lng = float(result['documents'][0]['x'])
  lat = float(result['documents'][0]['y'])
  lng_list.append(lng)
  lat_list.append(lat)

In [12]:
df['경도']= lng_list
df['위도']= lat_list
df

,공공기관,도로명주소,경도,위도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),126.978988,37.573505
1,노원구청,서울특별시 노원구 노해로 437(상계동),127.056297,37.654519
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),127.105860,37.514477
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),126.903357,37.563426
4,양천구청,서울특별시 양천구 목동동로 105(신정동),126.866543,37.517075


In [ ]:
df.to_csv('공공기관2.csv', index=False)

In [14]:
import folium

In [15]:
map =folium.Map(location=[df.위도.mean(),df.경도.mean()],zoom_start=11)
for i in df.index[:100]:
  folium.Circle(
      radius =200,
      location= [df.위도[i],df.경도[i]],    # 위도, 경도
      popup= folium.Popup(df.도로명주소[i], max_width=200),
      tooltip= df.공공기관[i], 
      color= '#3186cc',
      fill= 'True',
      fill_color='#3186cc'
  ).add_to(map)

title= '<h3 align="center" style="font-size:20px">서울시 구청 안내</h3>'
map.get_root().html.add_child(folium.Element(title))
map